<a href="https://colab.research.google.com/github/mr7495/image-classification-spatial/blob/main/MIT_Xception_GAP_512.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi #show GPU type

In [2]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
import cv2
import zipfile
import shutil
import random
import pandas as pd
import csv
import os

In [ ]:
#download MIT Indoors Scenes dataset from Kaggle (https://www.kaggle.com/itsahmad/indoor-scenes-cvpr-2019)
#Get a new download like from kaggle website at the mentioned URL, then replace the link with the "kaggle_linke" input in the next line of code 

!wget -cO - kaggle_link > mit_indoor_scenes.zip #replace kaggle_link with a new download link from https://www.kaggle.com/itsahmad/indoor-scenes-cvpr-2019

In [4]:
archive = zipfile.ZipFile('mit_indoor_scenes.zip') #Unzip MIT Indoors Scenes Dataset
for file in archive.namelist():
     archive.extract(file, 'data')

In [5]:
#Read dataset info
with open('data/TestImages.txt','r') as f:
  test_data=f.readlines()
for index,item in enumerate(test_data):
  if '\n' in item:
    test_data[index]=item[:-1]

with open('data/TrainImages.txt','r') as f:
  train_data=f.readlines()
for index,item in enumerate(train_data):
  if '\n' in item:
    train_data[index]=item[:-1]

In [6]:
#write train and test CSV files
with open('data/train.csv','w',newline='') as f:
  csvw=csv.writer(f)
  csvw.writerow(['filename','class'])
  for item in train_data:
    class_name=item[:item.index('/')]
    img_name='indoorCVPR_09/Images/'+item
    csvw.writerow([img_name,class_name])

with open('data/test.csv','w',newline='') as f:
  csvw=csv.writer(f)
  csvw.writerow(['filename','class'])
  for item in test_data:
    class_name=item[:item.index('/')]
    img_name='indoorCVPR_09/Images/'+item
    csvw.writerow([img_name,class_name])

In [7]:
#Set data augmentation techniques
train_datagen = keras.preprocessing.image.ImageDataGenerator(horizontal_flip=True,vertical_flip=True
                                                             ,zoom_range=0.2,rotation_range=360
                                                             ,width_shift_range=0.1,height_shift_range=0.1
                                                             ,channel_shift_range=50
                                                             ,brightness_range=(0,1.2)
                                                             ,preprocessing_function=keras.applications.imagenet_utils.preprocess_input)

test_datagen = keras.preprocessing.image.ImageDataGenerator(preprocessing_function=keras.applications.imagenet_utils.preprocess_input)

train_df = pd.read_csv("data/train.csv")
test_df = pd.read_csv("data/test.csv")

In [8]:
#Create Data augmentation techniques
batch_size=15
train_generator = train_datagen.flow_from_dataframe(
      dataframe=train_df,
      directory='data',
      x_col="filename",
      y_col="class",
      target_size=(512, 512),
      batch_size=batch_size,
      class_mode='categorical',shuffle=True)
validation_generator = test_datagen.flow_from_dataframe(
        dataframe=test_df,
        directory='data',
        x_col="filename",
        y_col="class",
        target_size=(512, 512),
        batch_size=batch_size,
        class_mode='categorical',shuffle=True)
train_img_num=len(train_generator.filenames)

Found 5360 validated image filenames belonging to 67 classes.
Found 1340 validated image filenames belonging to 67 classes.


In [ ]:
name="MIT-Xception-GAP-512"
!mkdir "models" #create new folder for saving checkpoints
!mkdir "reports" #create new folder for saving evaluation reports
keras.backend.clear_session() #clear backend
shape=(512,512,3)
input_tensor=keras.Input(shape=shape)
base_model=keras.applications.Xception(input_tensor=input_tensor,weights=None,include_top=False)
avg=keras.layers.GlobalAveragePooling2D()(base_model.output)
preds=keras.layers.Dense(67,activation='softmax',
                          kernel_initializer=keras.initializers.RandomNormal(mean=0.0,stddev=0.01),
                          bias_initializer=keras.initializers.Zeros(),)(avg)
model=keras.Model(inputs=base_model.input, outputs=preds)  

##################################
for layer in model.layers:
  layer.trainable = True
model.summary()
filepath="models/%s-{epoch:02d}-{val_accuracy:.4f}.hdf5"%name
checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor='val_accuracy', save_best_only=False, mode='max') #creating checkpoint to save the best validation accuracy
callbacks_list = [checkpoint]

#Determine adaptive learning rate with an initialization value of 0.045 and decay of 0.94 every two epochs.
lr_schedule =keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.045,
    decay_steps=2*int(train_img_num/batch_size),
    decay_rate=0.94,
    staircase=True)
optimizer=keras.optimizers.SGD(momentum=0.9,learning_rate=lr_schedule)
model.compile(optimizer=optimizer, loss='categorical_crossentropy',metrics=['accuracy'])
hist=model.fit_generator(train_generator, epochs=130,validation_data=validation_generator,shuffle=True,callbacks=callbacks_list) #start training
with open('reports/{}.csv'.format(name), mode='w',newline='') as csv_file: #write reports
  csv_writer = csv.writer(csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
  for key in hist.history:
    data=[key]
    data.extend(hist.history[key])
    csv_writer.writerow(data)
print("Training finished. Reports saved!")